Mapmatching
====================
Applies lane edits and parking spots to the street graph

Prerequisite: Preparing the simplified street graph

In [ ]:
import snman
from snman.constants import *

PERIMETER = '_debug'

# Set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data_v2/'
inputs_path = data_directory + 'inputs/'
process_path = data_directory + 'process/' + PERIMETER + '/'

#export_path = data_directory + 'outputs/' + 'matsim_zrh5 v5' + '/'
export_path = data_directory + 'mapmatching_test_output/' + PERIMETER + '/'

#CRS_internal = 29119    # for Boston
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [ ]:
print('Load street graph')
G = snman.io.load_street_graph(
    process_path + 'street_graph_edges.gpkg',
    process_path + 'street_graph_nodes.gpkg',
    crs=CRS_internal
)

In [ ]:
import geopandas as gpd
import shapely as shp

# load parking spaces
parking_spots = snman.io.load_parking_spots(inputs_path + 'zurich/strassenparkplaetze.gpkg', crs=CRS_internal)

# import source geometries for mapmatching
lane_edits = snman.io.load_lane_edits(inputs_path + 'boston/lane_edits.gpkg', crs=CRS_internal)


In [ ]:
lane_edits.head()

In [ ]:
snman.enrichment.match_lane_edits(G, lane_edits)

In [ ]:
snman.enrichment.match_parking_spots(G, parking_spots)

Export
------
Saves the resulting datasets to the disk. Use the *snman_detailed.qgz* file to view them in QGIS.

In [ ]:
if 1:
    print('Export network without lanes')
    snman.io.export_street_graph(
        G,
        export_path + 'street_graph_edges.gpkg',
        export_path + 'street_graph_nodes.gpkg',
        crs=CRS_for_export
    )

if 1:
    print('Export lane geometries')
    SCALING = 1
    snman.io.export_street_graph_with_lanes(
        G,
        KEY_LANES_DESCRIPTION,
        export_path + 'lane_geometries.shp',
        scaling=SCALING,
        crs=CRS_for_export
    )
    snman.io.export_street_graph_with_lanes(
        G,
        KEY_LANES_DESCRIPTION_AFTER,
        export_path + 'lane_geometries_after.shp',
        scaling=SCALING,
        crs=CRS_for_export
    )

if 0:
    print('Export lane graphs')
    L_before = snman.street_graph.to_lane_graph(G, KEY_LANES_DESCRIPTION)
    L_after = snman.street_graph.to_lane_graph(G, KEY_LANES_DESCRIPTION_AFTER)
    snman.io.export_street_graph(
        L_before,
        export_path + 'lane_graph_edges.gpkg', export_path + 'lane_graph_nodes.gpkg',
        crs=CRS_for_export
    )
    snman.io.export_street_graph(
        L_after,
        export_path + 'lane_graph_after_edges.gpkg', export_path + 'lane_graph_after_nodes.gpkg',
        crs=CRS_for_export
    )

if 0:
    print('Export OSM XML')
    snman.io.export_osm_xml(G, export_path + 'new_network.osm', EXPORT_OSM_TAGS, uv_tags=True)